In [8]:
import os
import numpy as np
import pandas as pd

In [2]:
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs


In [3]:
from config import settings

In [4]:

where_is = WhereIs(False)

In [5]:
all_paths = where_is.get_paths_list

In [6]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0], sample_ratio=0.2)


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
customers_df = load_customers(all_paths[1], sample_ratio=0.3)

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
test_file, lnr = load_test_file(all_paths[6])

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
clean_up = CleanUp(all_paths)



In [10]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df, 'test_file': test_file}

In [11]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_dict = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[5], info_df=all_paths[9],attr_df=all_paths[8])



/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].replace(value, np.nan)


In [12]:
#Run this step just one time
path_to_save = '../data'
#clean_azdias, clean_customer, clean_test
#Savindo version of data
#argument_dict['azdias'].to_csv(path_to_save + '/cleaned_after_pipeline/population.csv')

In [13]:
## create test column
argument_dict['azdias']['test_column'] = np.nan


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
argument_dict['azdias'].to_csv(path_to_save + '/cleaned_after_pipeline/population.csv')


In [1]:
#Open Data With DVC
import dvc.api
data_path='data/cleaned_after_pipeline/population.csv'
repo='/home/jair/Documents/arvato_project/'
version='population-v1'
data_url_v1 = dvc.api.get_url(
    path=data_path,
    repo=repo,
    rev=version
)

In [2]:
data_path

'data/cleaned_after_pipeline/population.csv'

In [3]:
version_2 = 'training-v2'
data_url_v2 = dvc.api.get_url(
    path=data_path,
    repo=repo,
    rev=version_2
)


In [6]:
data_v1 = pd.read_csv(data_url_v1)


FileNotFoundError: [Errno 2] No such file or directory: '../../../../../tmp/dvc-storage/64/c6d9e8161e5ef0d5d8a9063b2a7de5'

In [8]:
data_v2 = pd.read_csv(data_url_v2)


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
from models.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool
import pandas as pd

In [12]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())


I see 1 GPU devices


In [13]:
df_to_model = pd.concat(
    [dfs_dict['customers_df'].assign(is_customer=1),
     dfs_dict['azdias'].assign(is_customer=0)]
)


In [14]:
cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

In [18]:
features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')

X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))



In [19]:
catboost = CatBoostClassifier(num_trees=40,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40)


In [20]:
catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)

CatBoostError: Invalid type for cat_feature[non-default value idx=2,feature_idx=10]=8.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
show_metrics_baseline(catboost, features=features, labels=labels)


In [ ]:
#Predict the testfile

